# Параметрические критерии

## Task 1
В одном из выпусков программы "Разрушители легенд" проверялось, действительно ли заразительна зевота. В эксперименте участвовало 50 испытуемых, проходивших собеседование на программу. Каждый из них разговаривал с рекрутером; в конце 34 из 50 бесед рекрутер зевал. Затем испытуемых просили подождать решения рекрутера в соседней пустой комнате.

Во время ожидания 10 из 34 испытуемых экспериментальной группы и 4 из 16 испытуемых контрольной начали зевать. Таким образом, разница в доле зевающих людей в этих двух группах составила примерно 4.4%. Ведущие заключили, что миф о заразительности зевоты подтверждён.

Можно ли утверждать, что доли зевающих в контрольной и экспериментальной группах отличаются статистически значимо? Посчитайте достигаемый уровень значимости при **альтернативе заразительности зевоты**, округлите до четырёх знаков после десятичной точки.

**$H_0:$** зевота не заразительна $\hat{p}_1 = \hat{p}_2$  
**$H_1:$** зевота заразительна $\hat{p}_1 > \hat{p}_2$

In [1]:
from scipy import stats

In [2]:
a, n1 = 10, 34
b, n2 = 4, 16

p1, p2 = a/n1, b/n2

P = (p1*n1 + p2*n2) / (n1+n2)
z = (p1 - p2) / (P * (1-P) * (1/n1 + 1/n2))**0.5

In [3]:
p_value = 1 - stats.norm.cdf(z)
round(p_value, 4)

0.3729

## Task 2
Имеются данные измерений двухсот швейцарских тысячефранковых банкнот, бывших в обращении в первой половине XX века. Сто из банкнот были настоящими, и сто — поддельными. На рисунке ниже показаны измеренные признаки:
![caption](files/CHF1000_2_back_horizontal.jpg)
Загрузите данные: **banknotes.txt**  

Отделите 50 случайных наблюдений в тестовую выборку с помощью функции sklearn.cross_validation.train_test_split (зафиксируйте random state = 1). На оставшихся 150 настройте два классификатора поддельности банкнот:

1. логистическая регрессия по признакам $X_1, X_2, X_3$ 
2. логистическая регрессия по признакам $X_4, X_5, X_6$ 

Каждым из классификаторов сделайте предсказания меток классов на тестовой выборке. Одинаковы ли доли ошибочных предсказаний двух классификаторов? Проверьте гипотезу, вычислите достигаемый уровень значимости. Введите номер первой значащей цифры (например, если вы получили $5.5\times10^{-8}$, нужно ввести 8).

In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [5]:
data = pd.read_csv('datasets/banknotes.txt', sep='\t')
data.head()

,X1,X2,X3,X4,X5,X6,real
0,214.8,131.0,131.1,9.0,9.7,141.0,1
1,214.6,129.7,129.7,8.1,9.5,141.7,1
2,214.8,129.7,129.7,8.7,9.6,142.2,1
3,214.8,129.7,129.6,7.5,10.4,142.0,1
4,215.0,129.6,129.7,10.4,7.7,141.8,1


In [6]:
X = data.drop('real', axis=1)
y = data['real']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=50, random_state=1)

In [8]:
features1 = ['X1', 'X2', 'X3']
features2 = ['X4', 'X5', 'X6']

In [9]:
regressor1 = LogisticRegression()
regressor1.fit(X_train[features1], y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [10]:
regressor2 = LogisticRegression()
regressor2.fit(X_train[features2], y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [11]:
predictions1 = regressor1.predict(X_test[features1])
predictions2 = regressor2.predict(X_test[features2])

In [12]:
wrong_predictions1 = predictions1 != y_test
wrong_predictions2 = predictions2 != y_test

In [13]:
p1 = accuracy_score(y_test, predictions1)
p1

0.8

In [14]:
p2 = accuracy_score(y_test, predictions2)
p2

0.98

**$H_0:$** доли предсказаний классифкаторов равны $\hat{p}_1 = \hat{p}_2$   
**$H_1:$** доли не равны $\hat{p}_1 \ne \hat{p}_2$ 

  $X_1$ \ $X_2$ | 1| 0 | $\sum$
  ------------- | -------------|
  1  | e | f | e + f
  0  | g | h | g + h
  $\sum$ | e + g| f + h | n  
  
$$ \hat{p}_1 = \frac{e + f}{n}$$

$$ \hat{p}_2 = \frac{e + g}{n}$$

$$ \hat{p}_1 - \hat{p}_2 = \frac{f - g}{n}$$

$$Z-статистика: Z({X_1, X_2}) = \frac{f - g}{\sqrt{f + g - \frac{(f-g)^2}{n}}}$$

In [15]:
def proportions_diff_z_stat_rel(sample1, sample2):
    sample = list(zip(sample1, sample2))
    n = len(sample)
    
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    return float(f - g) / np.sqrt(f + g - float((f - g)**2) / n )

In [16]:
z_stat = proportions_diff_z_stat_rel(wrong_predictions1, wrong_predictions2)

In [17]:
p_value = 2*(1 - stats.norm.cdf(abs(z_stat)))
p_value

0.0032969384555543435

## Task 3
В предыдущей задаче посчитайте 95% доверительный интервал для разности долей ошибок двух классификаторов. Чему равна его ближайшая к нулю граница? Округлите до четырёх знаков после десятичной точки.


$$\text{Доверительный интервал для }p_1 - p_2\colon \;\;  \frac{f - g}{n} \pm z_{1-\frac{\alpha}{2}}\sqrt{\frac{f + g}{n^2} - \frac{(f - g)^2}{n^3}}$$

In [18]:
def proportions_diff_confint_rel(sample1, sample2, alpha = 0.05):
    z = stats.norm.ppf(1 - alpha / 2.)
    sample = list(zip(sample1, sample2))
    n = len(sample)
        
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    left_boundary = float(f - g) / n  - z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    right_boundary = float(f - g) / n  + z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    return (left_boundary, right_boundary)

In [19]:
left, right = proportions_diff_confint_rel(wrong_predictions1, wrong_predictions2)
left, right

(0.059945206279614305, 0.3000547937203857)

In [20]:
round(left, 4)

0.0599

## Task 4
Ежегодно более 200000 людей по всему миру сдают стандартизированный экзамен GMAT при поступлении на программы MBA. Средний результат составляет 525 баллов, стандартное отклонение — 100 баллов.

Сто студентов закончили специальные подготовительные курсы и сдали экзамен. Средний полученный ими балл — 541.4. Проверьте гипотезу о неэффективности программы против односторонней альтернативы о том, что программа работает. Отвергается ли на уровне значимости 0.05 нулевая гипотеза? Введите достигаемый уровень значимости, округлённый до 4 знаков после десятичной точки.



**$H_0:$** программа неэффективна $\mu = \mu_0$  
**$H_1:$** программа работает $\mu > \mu_0$  

In [21]:
mu_0, sigma = 525, 100
mean = 541.4
n = 100

In [22]:
z_stat = (mean-mu_0)/(sigma/np.sqrt(n))
p_value = 1 - stats.norm.cdf(z_stat)

In [23]:
round(p_value, 4)

0.0505

## Task 5
Оцените теперь эффективность подготовительных курсов, средний балл 100 выпускников которых равен 541.5. Отвергается ли на уровне значимости 0.05 та же самая нулевая гипотеза против той же самой альтернативы? Введите достигаемый уровень значимости, округлённый до 4 знаков после десятичной точки.

In [24]:
mean2 = 541.5
z_stat = (mean2-mu_0)/(sigma/np.sqrt(n))
p_value = 1 - stats.norm.cdf(z_stat)
round(p_value, 4)

0.0495